In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/sales-modified-data-in-pickle-1-grid-shop-items/"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
from itertools import product

In [ ]:
all_data = pd.read_pickle('../input/sales-modified-data-in-pickle-1-grid-shop-items/all_data.pkl')

In [ ]:
all_data.head(10)

In [ ]:
all_data['target'].sum() / all_data.shape[0]

- Create target mean encoding feature with CV loop regularization using KFold

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=False)

all_data['item_target_enc'] = np.NaN

for rest, curr in kf.split(all_data):
    # divide the data using folds
    #rest_fold, cur_fold = all_data[rest], all_data[curr]
    
    # use rest_fold to calculate mean target
    #all_data.iloc[curr]['item_target_enc'] = all_data.iloc[rest].groupby('item_id')['item_cnt_month'].transform('mean')
    # 8e-16
    # should be all_data.iloc[curr, all_data.columns.get_loc('item_target_enc')]
    
    #item_id_target_mean = all_data.iloc[rest].groupby('item_id').item_cnt_month.mean()
    #all_data.ix[curr,'item_target_enc'] = all_data['item_id'].map(item_id_target_mean)
    #.ix is depreciated
    # 0.41650796922
    
    # mean encodind item_id
    item_id_target_mean = all_data.iloc[rest].groupby('item_id').target.mean()
    all_data.loc[all_data.index[curr],'item_target_enc'] = all_data['item_id'].map(item_id_target_mean)
    # 0.4164590712798811
    
    # shop_id
    shop_id_target_mean = all_data.iloc[rest].groupby('shop_id').target.mean()
    all_data.loc[all_data.index[curr],'shop_target_enc'] = all_data['shop_id'].map(shop_id_target_mean)
    
    # item_category_id
    item_category_id_target_mean = all_data.iloc[rest].groupby('item_category_id').target.mean()
    all_data.loc[all_data.index[curr],'item_category_target_enc'] = all_data['item_category_id'].map(item_category_id_target_mean)
    
    # date_block_num
    date_block_num_target_mean = all_data.iloc[rest].groupby('date_block_num').target.mean()
    all_data.loc[all_data.index[curr],'date_block_num_target_enc'] = all_data['date_block_num'].map(date_block_num_target_mean)
    
    
    #all_data.ix[curr, 'item_target_enc'] = all_data.iloc[rest].groupby('item_id')['item_cnt_month'].transform('mean')
    # 0.0801429988734


In [ ]:
# Fill NaNs
all_data.fillna(0.33427, inplace=True)

# compute correlation
print(np.corrcoef(all_data['target'].values, all_data['item_target_enc'].values)[0][1])
print(np.corrcoef(all_data['target'].values, all_data['shop_target_enc'].values)[0][1])
print(np.corrcoef(all_data['target'].values, all_data['item_category_target_enc'].values)[0][1])
print(np.corrcoef(all_data['target'].values, all_data['date_block_num_target_enc'].values)[0][1])

In [ ]:
all_data.to_pickle('all_data2_mean_enc.pkl')